In [67]:
import pandas as pd
import numpy as np

import mlflow
from hyperopt import hp, STATUS_OK, fmin, Trials, tpe
from hyperopt.pyll import scope

from matplotlib import pyplot as plt
from IPython.display import display
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split, KFold, RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb

from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('Telcom Churn')
mlflow.sklearn.autolog(True)

2023/07/01 10:07:52 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/07/01 10:07:52 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


In [21]:
def load_data(path):
    data = pd.read_csv(path)
    data.columns = data.columns.str.replace(' ', '_').str.lower()

    categorical_col = data.dtypes[data.dtypes == 'object'].index.tolist()
    for col in categorical_col:
        data[col] = data[col].str.replace(' ', '_').str.lower()

    data = data[data['totalcharges'] != '_']
    data['totalcharges'] = data['totalcharges'].astype('float32')
    return data

In [22]:
def prepare_data(data):
    data['churn'] = (data.churn=='yes').astype(int)
    categorical_col = data.dtypes[data.dtypes == 'object'].index.tolist()
    numerical_col = ['tenure', 'totalcharges', 'monthlycharges']

    categorical_col.remove('customerid')
    train_data, test_data = train_test_split(data, test_size=0.25,
                                            random_state=0)

    train_x = train_data.drop(['churn'], axis = 1)
    test_x = test_data.drop(['churn'], axis = 1)

    train_y = train_data.pop('churn')
    test_y = test_data.pop('churn')

    dv = DictVectorizer(sparse = False)
    dv.fit(train_x[categorical_col + numerical_col].to_dict(orient = 'records'))

    train_x = dv.transform(train_x[categorical_col + numerical_col].to_dict(orient = 'records'))
    test_x = dv.transform(test_x[categorical_col + numerical_col].to_dict(orient = 'records'))
    out = (train_x, train_y, test_x, test_y, dv)
    return out

In [29]:
def log_evaluation(y_true, y_pred):

    accuracy_ = accuracy_score(y_true, y_pred)
    precision_ = precision_score(y_true, y_pred)
    recall_ = recall_score(y_true, y_pred)
    f1score_ = f1_score(y_true, y_pred)

    out = {"test_accuracy_score" : accuracy_, 
    "test_precision_score" :precision_, 
    "test_recall_score" : recall_, 
    "test_f1_score" : f1score_}
    return out

def mlflow_logging(model, model_name, params, eval, 
                   model_tag = None, developer = 'Godwin'):

    train_output_eval, test_output_eval = eval
    with mlflow.start_run():
            
        mlflow.set_tag('Developer', developer)
        mlflow.set_tag('Model Type', model_tag)
        mlflow.set_tag('Model Name', model_name)
        mlflow.log_params(params)
        
        mlflow.log_metrics(train_output_eval)
        mlflow.log_metrics(test_output_eval)
        mlflow.sklearn.log_model(model)
        
        mlflow.log_artifact('new_data.csv', 'data.csv')

In [35]:
path = './data/Telco-Customer-Churn.csv'
data = load_data(path)
train_x, train_y, test_x, \
        test_y, dv = prepare_data(data)

# Linear Model
c_values = range(1, 100, 10)
for c_value in c_values:

    with mlflow.start_run():
    
        mlflow.set_tag('Developer', 'Godwin')

        model = LogisticRegression(C = c_value)
        model.fit(train_x, train_y)

        test_pred = model.predict(test_x)
        test_output_eval = log_evaluation(test_y, test_pred)
        mlflow.log_metrics(test_output_eval)

2023/07/01 11:00:56 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'
2023/07/01 11:01:01 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'
2023/07/01 11:01:05 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'
2023/07/01 11:01:08 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'
2023/07/01 11:01:12 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'
2023/07/01 11:01:15 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'
2023/07/01 11:01:19 WARNING mlflow.sklearn: Failed t

In [39]:
def single_tree_objective(params):
    with mlflow.start_run():

        mlflow.set_tag('Developer', 'Godwin')

        model = DecisionTreeClassifier(**params)
        model.fit(train_x, train_y)

        test_pred = model.predict(test_x)
        test_output_eval = log_evaluation(test_y, test_pred)   
        
        mlflow.log_metrics(test_output_eval)
        
    return {"loss": -test_output_eval['test_accuracy_score'], 'status': STATUS_OK}

def random_forest_objective(params):
    with mlflow.start_run():

        mlflow.set_tag('Developer', 'Godwin')

        model = RandomForestClassifier(**params)
        model.fit(train_x, train_y)

        test_pred = model.predict(test_x)
        test_output_eval = log_evaluation(test_y, test_pred)   
        mlflow.log_metrics(test_output_eval)
        
    return {"loss": -test_output_eval['test_accuracy_score'], 'status': STATUS_OK}

def single_tree():

    space = {"max_depth": hp.randint("max_depth", 1, 15),
            'min_samples_split': hp.randint("min_samples_split", 2, 15),
            'min_samples_leaf': hp.randint("min_samples_leaf", 1, 15),
            "criterion": hp.choice("criterion", ["gini", "entropy"]),
            }

    best_result = fmin(fn= single_tree_objective,
                        space=space,
                        algo=tpe.suggest,
                        max_evals=50,
                        trials=Trials()
                        )
    return best_result

def random_forest(): 

    space = {"n_estimators": hp.choice("n_estimators", [100, 200, 300, 400,500,600]),
             'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
             'min_samples_split': hp.randint("min_samples_split", 2, 15),
             'min_samples_leaf': hp.randint("min_samples_leaf", 1, 15),
             "criterion": hp.choice("criterion", ["gini", "entropy"]),
             }

    best_result = fmin(fn=random_forest_objective,
                        space=space,
                        algo=tpe.suggest,
                        max_evals=50,
                        trials=Trials()
                        )
    return best_result


In [37]:
single_tree()

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

2023/07/01 11:01:32 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'



  2%|▏         | 1/50 [00:03<02:51,  3.50s/trial, best loss: -0.7792946530147895]

2023/07/01 11:01:35 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'



  4%|▍         | 2/50 [00:07<03:03,  3.82s/trial, best loss: -0.7792946530147895]

...

2023/07/01 11:04:22 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'



 98%|█████████▊| 49/50 [02:54<00:03,  3.45s/trial, best loss: -0.7986348122866894]

2023/07/01 11:04:26 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'



100%|██████████| 50/50 [02:57<00:00,  3.55s/trial, best loss: -0.7986348122866894]


{'criterion': 1, 'max_depth': 7, 'min_samples_leaf': 9, 'min_samples_split': 5}

In [40]:
random_forest()

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

2023/07/01 11:05:30 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'



  2%|▏         | 1/50 [00:09<07:24,  9.06s/trial, best loss: -0.8065984072810012]

2023/07/01 11:05:39 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'



  4%|▍         | 2/50 [00:20<08:18, 10.39s/trial, best loss: -0.8100113765642776]

...

2023/07/01 11:11:33 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'



 98%|█████████▊| 49/50 [06:08<00:06,  6.61s/trial, best loss: -0.8156996587030717]

2023/07/01 11:11:39 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'



100%|██████████| 50/50 [06:19<00:00,  7.58s/trial, best loss: -0.8156996587030717]


{'criterion': 1,
 'max_depth': 59.0,
 'min_samples_leaf': 5,
 'min_samples_split': 3,
 'n_estimators': 0}

In [42]:
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

In [53]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.test_f1_score >0.6",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.test_f1_score ASC"]
)

In [54]:
for run in runs:
    print(f"run id: {run.info.run_id}, F1 Score: {run.data.metrics['test_f1_score']:.4f}")

run id: b5eb75916ba045bc99b4902ea507a319, F1 Score: 0.6012


In [55]:
run_id = "b5eb75916ba045bc99b4902ea507a319"

model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="Custormer-churn-models")

model_name = "Custormer-churn-models"
latest_versions = client.get_latest_versions(name=model_name)

2023/07/01 11:26:21 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/07/01 11:26:21 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
Successfully registered model 'Custormer-churn-models'.
2023/07/01 11:26:21 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: Custormer-churn-models, version 1
Created version '1' of model 'Custormer-churn-models'.


In [57]:
for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: None


In [58]:
model_version = 1
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1688225181126, current_stage='Staging', description=None, last_updated_timestamp=1688226020087, name='Custormer-churn-models', run_id='b5eb75916ba045bc99b4902ea507a319', run_link=None, source='/home/godwin/Documents/Workflow/Churn-Prediction-in-a-Telecom-Company/mlruns/1/b5eb75916ba045bc99b4902ea507a319/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [59]:
from datetime import datetime

date = datetime.today().date() 

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1688225181126, current_stage='Staging', description='The model version 1 was transitioned to Staging on 2023-07-01', last_updated_timestamp=1688226074603, name='Custormer-churn-models', run_id='b5eb75916ba045bc99b4902ea507a319', run_link=None, source='/home/godwin/Documents/Workflow/Churn-Prediction-in-a-Telecom-Company/mlruns/1/b5eb75916ba045bc99b4902ea507a319/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [ ]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df



def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [64]:
import mlflow
logged_model = 'runs:/b5eb75916ba045bc99b4902ea507a319/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# # Predict on a Pandas DataFrame.
# import pandas as pd
# loaded_model.predict(pd.DataFrame(data))

2023/07/01 12:04:20 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.4.1, required: mlflow==2.4)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


In [63]:
loaded_model.predict(train_x)

array([0, 0, 1, ..., 0, 0, 0])

In [62]:
loaded_model

'runs:/b5eb75916ba045bc99b4902ea507a319/model'

In [65]:
client.delete_model_version(model_name, 1)

In [66]:
client.delete_registered_model(model_name)

In [ ]:
train = xgb.DMatrix(train_x, label=train_y)
valid = xgb.DMatrix(test_x, label=test_y)

def xgboost_objective(params):
    with mlflow.start_run():

        mlflow.set_tag('Developer', 'Godwin')
        mlflow.set_tag('Model Type', 'Base Model')
        mlflow.set_tag("model", "Xgboost")
        mlflow.log_params(params)
        model = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        train_pred = model.predict(train)
        train_output_eval = evaluation(train_y, train_pred, 'train')
        test_pred = model.predict(valid)
        test_output_eval = evaluation(test_y, test_pred, 'test')   
        data.to_csv('new_data.csv', header=False)    
        mlflow.log_metrics(train_output_eval)
        mlflow.log_metrics(test_output_eval)
        mlflow.log_artifact('new_data.csv', 'data.csv')
    return {'loss': test_output_eval['Test accuracy Score'], 'status': STATUS_OK}